In [1]:
import requests
import pandas as pd 
import json
from dateutil import parser

In [3]:
API_KEY = "00d239aeae8ecbaf50a4c1352c1837eb-73fbde84df7b82be7a15845f729b3abe"
ACCOUNT_ID = "101-004-30986958-001"
OANDA_URL= "https://api-fxpractice.oanda.com/v3"

In [2]:
session = requests.Session()

In [4]:
session.headers.update({
"Authorization": f"Bearer {API_KEY}",
"Content-Type": "application/json"
})

In [5]:
params = dict(
    count = 10,
    granularity = "H1",
    price = "MBA"
    
)

In [6]:
url = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/instruments"

In [7]:
response = session.get(url, params = params, data = None, headers = None) 

In [8]:
response.status_code 

200

In [9]:
data = response.json()

In [10]:
instruments_list = data['instruments']

In [11]:
len(instruments_list)

123

In [12]:
instruments_list[0].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'tags', 'financing'])

In [13]:
key_i = ['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'tags', 'financing']

In [ ]:
instruments_dict = {}
for i in instruments_list:
    key = i['name']
    instruments_dict[key] = {k: i[k] for k in key_i }

In [15]:
instruments_dict['USD_CAD']

{'name': 'USD_CAD',
 'type': 'CURRENCY',
 'displayName': 'USD/CAD',
 'pipLocation': -4,
 'displayPrecision': 5,
 'tradeUnitsPrecision': 0,
 'minimumTradeSize': '1',
 'maximumTrailingStopDistance': '1.00000',
 'minimumTrailingStopDistance': '0.00050',
 'maximumPositionSize': '0',
 'maximumOrderUnits': '100000000',
 'marginRate': '0.03333333333333',
 'guaranteedStopLossOrderMode': 'ALLOWED',
 'tags': [{'type': 'BRAIN_ASSET_CLASS', 'name': 'FX'},
  {'type': 'ASSET_CLASS', 'name': 'CURRENCY'}],
 'financing': {'longRate': '0.0045',
  'shortRate': '-0.0251',
  'financingDaysOfWeek': [{'dayOfWeek': 'MONDAY', 'daysCharged': 1},
   {'dayOfWeek': 'TUESDAY', 'daysCharged': 1},
   {'dayOfWeek': 'WEDNESDAY', 'daysCharged': 1},
   {'dayOfWeek': 'THURSDAY', 'daysCharged': 1},
   {'dayOfWeek': 'FRIDAY', 'daysCharged': 1},
   {'dayOfWeek': 'SATURDAY', 'daysCharged': 0},
   {'dayOfWeek': 'SUNDAY', 'daysCharged': 0}]}}

In [16]:
# -4 -> 0.0001
pow(10, -4)

0.0001

In [17]:
with open("../data/instrument.json", "w") as f:
    f.write(json.dumps(instruments_dict, indent=2))

In [28]:
def fetch_candles(pair_name, count=10, granularity="H1"):
    url = f"{OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"  
        
    )
    response = session.get(url, params=params, data=None, headers=None)
    data = response.json()
    
    if response.status_code == 200:
        if 'candles' not in data:
            data = []
        else:
            data = data['candles']
    return response.status_code, data

def get_candles_df(data):
    if len(data) == 0:
        return pd.DataFrame.empty
    
    prices = ['mid', 'bid', 'ask']
    ohlc = ['o', 'h', 'l', 'c']
    
    final_data = []
    for candle in data:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = parser.parse(candle['time'])
        new_dict['volume'] = candle['volume']
        for p in prices:
            for o in ohlc:
                new_dict[f"{p}_{o}"] = float(candle[p][o])
        final_data.append(new_dict)
    df = pd.DataFrame.from_dict(final_data)
    return df


def create_data_file(pair_name, count=10, granularity="H1"):
    code, data = fetch_candles(pair_name, count, granularity)
    if code != 200:
        print("Failed", pair_name, data)
        return
    if len(data) == 0:
        print("No candles", pair_name)
    candles_df = get_candles_df(data)
    candles_df.to_pickle(f"../data/{pair_name}_{granularity}.pkl")
    print(f"{pair_name} {granularity} {candles_df.shape[0]} candles, {candles_df.time.min()} {candles_df.time.max()}")
    
    

In [24]:
Code, data = fetch_candles("EUR_USD", count=10, granularity= "H4")
get_candles_df = get_candles_df(data)

In [30]:
create_data_file("EUR_USD", count=10, granularity="H4")

EUR_USD H4 9 candles, 2025-02-10 02:00:00+00:00 2025-02-11 10:00:00+00:00


In [29]:
our_curr = ['EUR', 'USD', 'GBP', 'JPY', 'CHF', 'NZD', 'CAD', 'AUD']

In [ ]:
for p1 in our_curr:
    for p2 in our_curr:
        pr = f"{p1}_{p2}"
        if pr in instruments_dict:
            for g in ["H1", "H4"]:
                create_data_file(pr, count=4001, granularity=g)